In [1]:
# Import libraries
from library.common import Database
import pathlib

import numpy as np
import pandas as pd
import os
import altair as alt
import warnings
warnings.filterwarnings("ignore")

# Display all columns of pandas dataframes
pd.set_option('display.max_columns', None)

db = Database()
write_to_db = True

# Get the path to raw data files
# path = pathlib.Path.cwd().parent.joinpath(path_to_data)

Read the master dataset from Database

In [2]:
data2 = pd.read_sql('master_dataset', db.config)
data2

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,consumption_co2_per_gdp,co2_per_unit_energy,coal_co2,cement_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,cement_co2_per_capita,coal_co2_per_capita,flaring_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,other_co2_per_capita,trade_co2_share,share_global_cement_co2,share_global_coal_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,cumulative_cement_co2,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_oil_co2,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,total_ghg,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp,population,current_gdp,constant_gdp,manufacturing_gdp,medium_to_high_tech_percent,export,import,real_gdp_growth_percent,urban_population_percent,merchandise_export,merchandise_import,manufacturer_export_share,manufacturer_export,co2_emission_electricity,co2_emission_building,co2_emission_manufacturing,co2_emission_other_fuel,co2_emission_fugitive,co2_emission_transport,co2_emission_energy_subtotal,co2_emission_bunkers,co2_emission_industrial_process,co2_emission_per_capita,constant_gdp_per_capita,manufacturing_percent,medium_to_high_tech_gdp,co2_emission_per_constant_gdp,trade_openness,share_of_merchandise_export,share_of_merchandise_import,industrial_gdp,co2_country_share_percent,manufacturing_country_share_percent,iea_primary_energy_consumption,renewable_energy_consumption,coal_consumption,oil_consumption,total_electricity_production,electricity_production_from_renewable,primary_energy_consumption_per_capita,fossil_energy_consumption_share,renewable_electricity_production_share,energy_intensity,renewable_energy_consumption_share,percent_of_environment_patent,region,income_group
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,NaN,0.015,NaN,NaN,NaN,NaN,0.015,NaN,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,NaN,0.099,NaN,0.009,NaN,NaN,0.021,NaN,NaN,NaN,0.063,NaN,NaN,0.003,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.036,NaN,NaN,0.063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.421400e+09,NaN,NaN,NaN,7752117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,NaN,0.191,NaN,0.010,NaN,NaN,0.026,NaN,NaN,NaN,0.066,NaN,NaN,0.003,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061,NaN,NaN,0.129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.692280e+09,NaN,NaN,NaN,7840151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,AFG,Afghanistan,1952,0.092,NaN,NaN,NaN,NaN,0.012,NaN,NaN,0.282,NaN,0.009,NaN,NaN,0.032,NaN,NaN,NaN,0.060,NaN,NaN,0.004,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093,NaN,NaN,0.189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.001732e+10,NaN,NaN,NaN,7935996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No

In [3]:
diff_lst=['MTQ','GUF','MSR','COK','BES','ATA','SHN','TWN','SPM','MYT','AIA','CXR','GLP','NIU']
data2=data2[data2['year']>=1960]
data2_world=data2[data2['iso_code']=='WLD']
data2=data2[data2['iso_code']!='WLD']
data2=data2[~data2['iso_code'].isin(diff_lst)]
data2=data2.sort_values(["year", "iso_code"], ascending = (True, True))
data2=data2.append(data2_world).reset_index()
data2.drop(['index'],axis=1,inplace=True)
data2['co2']=data2['co2']*1000
data2['constant_gdp']=data2['constant_gdp_per_capita']*data2['population']
data2['primary_energy_consumption']=data2['primary_energy_consumption_per_capita']*data2['population']
data2['renewable_energy_consumption_per_capita']=(data2['primary_energy_consumption']*0.000000041868*(data2['renewable_energy_consumption_share']/100))/data2['population']

## Step 2: Exploratory Data Analysis: Carbon dioxide emission 1960-2020

### Observation 1: Total carbon dioxide emission is on increasing trend, but may decline when there were economic recessions

On global basis, carbon dioxide emission was about 10 billion ton in 1960.  It increased by over 2 times to 35 billion ton in 2020.  While carbon dioxide emission basically increased every year, we observed occasional year-on-year decrease when there were economic recessions.  For example, we observed the largest year-on-year decline in emission level over the past 60 years, at -5% in 2020 when many goverments imposed restrictions on domestic and international travels, cut operating hours of bars and restaurants, required office implement work-from-home arrangements, etc. to curb the spread of COVID-19 pandemic. As a result, many countries suffered the worst economic downturns since World War II.

In [4]:
data2_world=data2[data2['iso_code']=='WLD']
data2_CHN=data2[data2['iso_code']=='CHN']
data2_USA=data2[data2['iso_code']=='USA']
data2_world['co2_emission_lag1']=data2_world['co2'].shift(1)
data2_world['co2_emission_yoy']=(data2_world['co2']/data2_world['co2_emission_lag1']-1)*100

In [5]:
# Annual Global CO2 emission
chart1a=alt.Chart(data2_world).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2:Q',
          axis=alt.Axis(title='CO2 Emission (thousand ton)')),
    color=alt.value('forestgreen')
).properties(width=400, height=300,
             title=alt.TitleParams(text='Annual Global CO2 Emission, 1960-2020',
                                   fontSize=15))

yoy=alt.Chart(data2_world).mark_bar().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2_emission_yoy:Q',
          axis=alt.Axis(title='Year-on-year change in CO2 emission (%)')),
    color=alt.condition('datum.co2_emission_yoy>=0', alt.ColorValue('cornflowerblue'), alt.ColorValue('lightcoral'))
)
text1 = alt.Chart({'values':[{'y': -6.05}]}).mark_text(
    text='COVID-19 \npandemic',dx=180,lineBreak='\n',size=10
).encode(
    y='y:Q'
)
text2 = alt.Chart({'values':[{'y': -1.9}]}).mark_text(
    text='Global \nfinancial \ncrisis',dx=120,lineBreak='\n',size=10
).encode(
    y='y:Q'
)
text3 = alt.Chart({'values':[{'y': -3.3}]}).mark_text(
    text='Gulf war',dx=10,size=10
).encode(
    y='y:Q'
)
text4 = alt.Chart({'values':[{'y': -2.7}]}).mark_text(
    text='Energy crisis \nafter Iranian\nrevolution',dx=-65,lineBreak='\n',size=10
).encode(
    y='y:Q'
)
chart1b = alt.layer(yoy,text1,text2,text3,text4).resolve_scale(y='shared').properties(width=400, height=300,
             title=alt.TitleParams(text='Year-on-year % Change in Global CO2 Emission',
                                   fontSize=15))

chart1a|chart1b

alt.HConcatChart(...)

### Observation 2: Electricity and heat is the major source of carbon dioxide emission

Over the past three decades from 1990 to 2018, electricity and heat was the largest source of carbon dioxide emission and represented slightly over 40% of total carbon dioxide emitted.  Manufacturing and construction came next and represented about 15% of total emission.

In [6]:
chart_a=data2_world[(data2_world['year']>=1990)&(data2_world['year']<=2018)]
chart_a=chart_a[['year','co2','co2_emission_electricity','co2_emission_building','co2_emission_manufacturing','co2_emission_transport']]
chart_a['co2_emission_other']=chart_a['co2']-chart_a['co2_emission_electricity']-chart_a['co2_emission_building']-chart_a['co2_emission_manufacturing']-chart_a['co2_emission_transport']
chart_a.rename(columns={'co2_emission_electricity':'Electricity & Heat','co2_emission_building':'Building',
                        'co2_emission_manufacturing':'Manufacturing & Construction','co2_emission_transport':"Transport (excl. int'l bunkers)",
                        'co2_emission_other':'Others'}, inplace=True)
chart_a.drop(['year','co2'],axis=1,inplace=True)
stacked_df=chart_a.stack().reset_index()
stacked_df.drop(['level_0'],axis=1,inplace=True)
stacked_df.rename(columns={'level_1':'Source',0:'Amount'}, inplace=True)

import itertools
lst = range(1990,2019)
year_lst=list(itertools.chain.from_iterable(itertools.repeat(x, 5) for x in lst))
stacked_df['year']=year_lst

In [7]:
bars = alt.Chart(stacked_df).mark_bar().transform_joinaggregate(
    order='sum(Amount)',
    groupby=['year']).encode(
    x=alt.X('sum(Amount):Q', stack='normalize', 
            axis=alt.Axis(title='Share of Emission Source (%)',format='%')),
    y=alt.Y('year:O',
            axis=alt.Axis(title='Year')),
    color=alt.Color('Source:N',
                     scale=alt.Scale(domain=['Electricity & Heat', 'Manufacturing & Construction', 'Building', "Transport (excl. int'l bunkers)", 'Others'],
                     range=['orange','lightskyblue','lightyellow','mediumturquoise','thistle'])),
    order=alt.Order('order:Q',sort='descending')
                     
).properties(width=500, height=250,
             title=alt.TitleParams(text='Percentage Share of CO2 Emission by Source',dy=-5,fontSize=15)
            )

bars 

alt.Chart(...)

### Observation 3: Decreasing share of North America and increasing share of Asia Pacific in total carbon dioxide emission were observed

In 1960, emission from Europe and North America represented nearly 80% of total global emission.  Over the past 60 years, share of Europe and North America shrank gradually, while share of Asia picked up.  In 2020, emission from Europe and North America represented only slightly over 30% of total global emission.  while share of East Asia and South Asia rose to over 50%.

In [8]:
data2_60_20=data2[~(data2['iso_code']=='WLD')]
data2_60_20gp=data2_60_20.groupby(['year','region']).agg({'co2':'sum'}).reset_index()

In [9]:
# Annual CO2 emission by region (don't know how to swap order)
#site_order = ['Europe & Central Asia', 'North America', 'Sub-Saharan Africa', 'Latin America & Caribbean', 'Middle East & North Africa', 'South Asia', 'East Asia & Pacific']
chart2=alt.Chart(data2_60_20gp).transform_joinaggregate(
    order='min(co2)',
    groupby=['region']).mark_area().encode(
           x=alt.X("year:O",
                   axis=alt.Axis(title='Year')
                  ),
           y=alt.Y("co2:Q",stack='normalize',
                   axis=alt.Axis(title='Share of CO2 Emission (%)',format='%')
                  ),
           color=alt.Color('region:N', 
                    scale = alt.Scale(domain=['Europe & Central Asia','North America','Sub-Saharan Africa','Latin America & Caribbean','Middle East & North Africa','South Asia','East Asia & Pacific'],
                                      range=['darkred','lightcoral','lightcyan','lightskyblue','cornflowerblue','royalblue','darkblue']),
                    legend = alt.Legend(title='Region')
                          ),
           order=alt.Order('order:Q', sort='ascending')
        ).properties(width=400, height=300,
                     title=alt.TitleParams(text='Percentage Share of CO2 Emission by Region',fontSize=15)
                    ) 
chart2

alt.Chart(...)

If we focus on the two largest emitting countries in the world, viz. the US and China, it will be easy to understand why share of North America decreased and share of Asia increased over the past six decades. In fact, the decreasing share of North America and increasing share of Asia basically reflected the decreasing share of the US and the increasing share of China over the same period.

During 1960 - 2005, the US was the largest emitting country in the world.  Emission from the US in 1960 was about 3 billion ton, represented 30% of global emission at that time.  Emission from the US in the next 45 years continued to climb up, but at a rate slower than the global average.  Consequently, share of US in the global emission gradually fell to about 20% in 2005, albeit the emission amount actually rose to over 6 billion ton.  Since then, emission amount from the US started to decrease. In 2020, US emitted slightly less than 5 billion ton or represented 14% of the global emission, and is the second largest carbon dioxide emitting country in the world.

Before 1980, emission amount from China was less than 1.5 billion ton each year. The Open Door Policy since 1980s attracted foreign investment and led to the first-stage expansion in China's economic ties with the West.  Carbon dioxide emission amount also started to pick up.  In 2000, emission amount from China already rose to about 3.5 billion ton, represented nearly 15% of global emission at that time.  The admission of China to World Trade Organization led to the second-stage expansion of China's economy. At the same time, carbon dioxide emission amount skyrocketed.  In only ten years, emission amount from China increased by 1.5 times and reached 9 billion ton in 2010, or over 25% of global emission at that time.  In fact, China has become the largest carbon dioxide emitting country in the world since 2006.  In 2020, emission amount from China was nearly 11 billion ton, represented over 30% of global emission.

In [10]:
# Annual CO2 emission - by two largest emitters (US & China)
data_chart3 = data2_USA.append(data2_CHN,ignore_index=True)
chart3a=alt.Chart(data_chart3).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2:Q',
          axis=alt.Axis(title='CO2 Emission (thousand ton)')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=380, height=300,
             title=alt.TitleParams(text='Annual CO2 Emission: US vs China',
                                   fontSize=15))

chart3b=alt.Chart(data_chart3).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2_country_share_percent:Q',
          axis=alt.Axis(title='Share of CO2 emission (%)')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=380, height=300,
             title=alt.TitleParams(text='Percentage share of CO2 Emission: US vs China',
                                   fontSize=15))
chart3a|chart3b

alt.HConcatChart(...)

The sharp rise in total carbon dioxide emission of China since 2000 seems tally with an accelerated rise in various factors which drive carbon dioxide emission as mentioned in various literature, including a steeper growth in real-term economic size (as measured by GDP at constant dollars) and energy consumption (as measured by primary energy consumption, which includes consumption of the energy sector, losses during transformation and distribution of energy, and the final consumption by end users); the becoming of the 'world's factory' (as measured by an increasing share in the total value added generated from manufacturing activities in the world); and a slightly faster increase in share of urban population compared to the growths observed in 1990s.

On the contrary, we observed that the US has become a more energy-efficient economy in tandem with a similar level of carbon dioxide emission between 1990 and 2020.  While the whole US economy has broadly consumed similar amount of energy every year over the past 30 years, the real-term economic size has continued to grow steadily over the same period. In other words, the US is able to consume less amount of energy in order to produce the same dollar amount of GDP. 

In [11]:
data_chart4 = data_chart3[data_chart3['year']>=1990]
data_chart4['constant_gdp']=data_chart4['constant_gdp']/1000000 #conver from $US to $US million
chart4a=alt.Chart(data_chart4).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('constant_gdp:Q',
          axis=alt.Axis(title='2015 $US million')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Constant GDP',
                                   fontSize=15))

chart4c=alt.Chart(data_chart4).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('primary_energy_consumption:Q',
          axis=alt.Axis(title='Petajoules')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Primary Energy Consumption',
                                   fontSize=15))

chart4d=alt.Chart(data_chart4).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('urban_population_percent:Q',
          axis=alt.Axis(title='Percent (%)')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=200, height=200,
             title=alt.TitleParams(text='Share of Urban Population in',subtitle='Total Population of the Country',
                                   fontSize=15,subtitleFontSize=15,subtitleFontWeight='bold'))
chart4=chart4a|chart4c|chart4d
chart4

alt.HConcatChart(...)

Since many products manufactured in China are produced on behalf of Western brand names in order to take advantage of the more cost efficient supply chain in China, some people argued that some of the carbon dioxide emission recorded in developing countries (like China) should actually been counted as emission in developed countries (like the US) which finally consumed the products.  Consequently, some experts have computed the trade-adjusted 'consumption-based carbon dioxide emission'.

Although carbon dioxide emission from the US increased and emission from China decreased under the consumption-based emission definition, China was still the largest emitting country in the world and the US comes second.

In [12]:
data_chart5 = data2[(data2['year']>=1990)&(data2['year']<=2018)]
data_chart5 = data_chart5[(data_chart5['iso_code']=='CHN')|(data_chart5['iso_code']=='USA')]
data_chart5['consumption_co2'] = data_chart5['consumption_co2']*1000
chart5a=alt.Chart(data_chart5).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('consumption_co2:Q',
          axis=alt.Axis(title='Thousand ton')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=500, height=300,
             title=alt.TitleParams(text='Annual CO2 Emission: Consumption-based vs Production-based',
                                   subtitle='Consumption-based emission in solid line',
                                   fontSize=15,subtitleFontSize=12))

chart5b=alt.Chart(data_chart5).mark_line(strokeDash=[3,1]).encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2:Q',
          axis=alt.Axis(title='CO2 emission (thousand ton)')),
    color=alt.Color('country:N',
                    scale = alt.Scale(domain=['China', 'United States'],
                                      range=['orange', 'cornflowerblue']),
                    legend = alt.Legend(title='Country')
                   )
).properties(width=500, height=300,
             title=alt.TitleParams(text='Annual CO2 Emission: Consumption-based vs Production-based',
                                   subtitle='Consumption-based emission in solid line',
                                   fontSize=15,subtitleFontSize=14))

text1 = alt.Chart({'values':[{'y': 7300000}]}).mark_text(
    text='US emitted more \nunder consumption-based',lineBreak='\n',dx=-10,size=10
).encode(
    y='y:Q'
)

text2 = alt.Chart({'values':[{'y': 8200000}]}).mark_text(
    text='China emitted less \nunder consumption-based',lineBreak='\n',dx=200,size=10
).encode(
    y='y:Q'
)

alt.layer(chart5a, chart5b, text1, text2).resolve_scale(
    x = 'shared',
    y = 'shared'
)

alt.LayerChart(...)

### Observation 4: Different trends were observed between total carbon dioxide emission and carbon dioxide emission intensity

While total carbon dioxide emission increased by 2.5 times from 10 billion ton to 35 billion over 1960-2020, the trend looks slightly different if we study 'emission per capita' and 'emission per constant GDP dollar' (i.e., carbon dioxide emission intensity).  Specifically, when expressing emission on per capita basis, carbon dioxide emission increased only by 45% over the past 60 years, from about 3.0 ton per person to 4.5 ton per person in 2020.  If we express emission on intensity basis, carbon dioxide emission actually decreased by 50%, from about 0.9kg per constant GDP dollar to about 0.4kg per constant GDP dollar over the same period.  This means that we actually emit less carbon dioxide in order to generate one GDP dollar!

We have introduced above that China and the US are the largest and the second largest countries in terms of total emission.  In terms of emission per capita, we may guess that Western countries will generally have higher amount of emission per person. This is quite true, but when only considering countries with population greater than 2 million, the US actually ranked 8th in 2020.  Qatar was the country which had the highest amount of emission per person in 2020.  In fact, top countries which have the highest amount of emission per person are usually oil producing countries, including Qatar, Kuwait and Saudi Arabia.

Carbon dioxide emission intensity is a bit difficult to understand.  Generally speaking, developed countries have more advanced and efficient production technology as well as higher proportion of service sectors in GDP, so they will have lower emission intensities, meaning that they can generate less amount of carbon dioxide per unit of economic output.  On the contrary, developing countries have less efficient production technology and lower proportion of service sectors in GDP, so they will have higher emission intensities, meaning that they need to generate more carbon dioxide per unit of economic output.  In 2020, Mongolia was the country with the highest emission intensity, and Ukraine was the second largest country.

In [13]:
data2_2020=data2[data2['year']==2020]
data2_2020=data2_2020[~(data2_2020['iso_code']=='WLD')]
data2_2020a=data2_2020.sort_values(by='co2',ascending=False).head(20)
data2_2020p=data2_2020[data2_2020['population']>=2000000]
data2_2020bp=data2_2020p.sort_values(by='co2_emission_per_capita',ascending=False).head(20)
data2_2020c=data2_2020.sort_values(by='co2_emission_per_constant_gdp',ascending=False).head(20)

In [14]:
chart5a=alt.Chart(data2_world).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2:Q',
          axis=alt.Axis(title='CO2 Emission (thousand ton)')),
    color=alt.value('forestgreen')
).properties(width=300, height=300,
             title=alt.TitleParams(text='Annual Global CO2 Emission, 1960-2020',
                                   fontSize=15))

chart5b=alt.Chart(data2_world).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2_emission_per_capita:Q',
          axis=alt.Axis(title='CO2 Emission (ton per person)')),
    color=alt.value('forestgreen')
).properties(width=300, height=300,
             title=alt.TitleParams(text='Global CO2 Emission per capita, 1960-2020',
                                   fontSize=15))

chart5c=alt.Chart(data2_world).mark_line().encode(
    alt.X('year:O', axis=alt.Axis(title='Year')),
    alt.Y('co2_emission_per_constant_gdp:Q',
          axis=alt.Axis(title='CO2 Emission (kg per $GDP (2015 $US))')),
    color=alt.value('forestgreen')
).properties(width=300, height=300,
             title=alt.TitleParams(text='Global CO2 Emission Intensity, 1960-2020',
                                   fontSize=15))

In [15]:
# Annual Global CO2 emission and CO2 emission per capita - Top 10 countries in 2020
chart5a2 = alt.Chart(data2_2020a).mark_bar().encode(
    x=alt.X('co2:Q',
            title='CO2 Emission (thousand ton)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=10)),
    y=alt.Y('country:N',
            sort='-x',
            title='',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    color=alt.Color('income_group:N', 
                    scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                    legend = alt.Legend(title='Income Group'))
).properties(width=250, height=300,
            title=alt.TitleParams(text='Annual CO2 Emission 2020',
                                   subtitle='The 20 countries with the highest amount of emission', subtitleFontSize=13,
                                   fontSize=15,dy=-5))

chart5b2 = alt.Chart(data2_2020bp).mark_bar().encode(
    x=alt.X('co2_emission_per_capita:Q',
            title='CO2 emission per capita (ton per person)',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('country:N',
            sort='-x',
            title='',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    color=alt.Color('income_group:N', 
                    scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                    legend = alt.Legend(title='Income Group'))
).properties(width=250, height=300,
            title=alt.TitleParams(text='CO2 Emission per capita 2020',
                                   subtitle='The 20 countries with the highest amount of emission', subtitleFontSize=13,
                                   fontSize=15,dy=-5))


chart5c2 = alt.Chart(data2_2020c).mark_bar().encode(
    x=alt.X('co2_emission_per_constant_gdp:Q',
            title='CO2 emission intensity (kg per $GDP (2015 $US))',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    y=alt.Y('country:N',
            sort='-x',
            title='',
            axis=alt.Axis(titleFontSize=12,labelFontSize=12)),
    color=alt.Color('income_group:N', 
                    scale = alt.Scale(domain=['High income', 'Upper middle income', 'Lower middle income', 'Low income'],
                                      range=['navy', 'cornflowerblue', 'lightcoral', 'darkred']),
                    legend = alt.Legend(title='Income Group'))
).properties(width=250, height=300,
            title=alt.TitleParams(text='CO2 Emission Intensity 2020',
                                   subtitle='The 20 countries with the highest intensity', subtitleFontSize=13,
                                   fontSize=15,dy=-5))
chart5_1=chart5a|chart5a2
chart5_2=chart5b|chart5b2
chart5_3=chart5c|chart5c2
chart5=alt.vconcat(chart5_1,chart5_2,chart5_3)
chart5

alt.VConcatChart(...)